# The annotated encoder-decoer with attention

Based on [this blog](https://bastings.github.io/annotated_encoder_decoder/)

Consider an input sequence $X=(x_1, ..., x_M)$ and a target sequence $Y=(y_1, ..., y_N)$. We will model the probability $p(Y|X)$ directly with a neural network: an encoder-decoder.

<img src="../figures/enc.jpg">

## Encoder

The encoder reads in the source sentence (bottom of above figure) and produces a sequence of hidden states $\mathbf{h_1,...,h_M}$ for each input word. These hidden states should capture the meaning of a word **in its context** given the sentence. Use a Bi-GRU as the encoder.

We first embed the source words: each source word's embedding is denoted as a vector $\mathbf{x}_i$. Using an embedding allows us to exploit the fact that certain words are semantically similar, and should therefore be processed in a similar way by having close embedding vectors.

We obtain the hidden states $\mathbf{h_1,...,h_M}$ from the recursive formula
$$\mathbf{h}_j = GRU(\mathbf{x}_j, \mathbf{h}_{j-1})$$
for a forward GRU. A backward GRU reads from right-to-left instead of left-to-right. We therefore obtain two hidden state vectors for each word. Concatenating the two hidden states together allows us to obtain a local context for each word.

## Decoder

The decoder is also a GRU with hidden state $\mathbf{s}_i$. It follows a similar formula to the encoder, but takes one extra input $\mathbf{c}_i$ (shown in yellow) called the **context**
$$\mathbf{s}_i = f(\mathbf{s}_{i-1}, \mathbf{y}_{i-1}, \mathbf{c}_{i})$$
where $\mathbf{y}_{i-1}$ is the embedding for the previously generated target word.

At each time step, an **attention mechanism** dynamically selects the part of the source sentence that is most relevant for predicting the current target word. It does so by comparing the last decoder state  with each source hidden state (**TODO: FILL IN MATHS**)

After computing the decoder state $\mathbf{s}_i$, a non-linear function $g$ (softmax) gives us the probability of the target word $y_i$ for this time step:
$$p(y_i|y_{<i},x_1^M) = g(\mathbf{s}_{i-1}, \mathbf{y}_{i-1}, \mathbf{c}_{i})$$

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [5]:
USE_CUDA = torch.cuda.is_available()
DEVICE=torch.device('cuda:0' if USE_CUDA else 'cpu')

In [9]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed);
if USE_CUDA:
    torch.cuda.manual_seed(seed)

## The model